# Introduction

In this workbook, we demonstrate the usage of LightGraphs to execute diffusion simulations on random graphs. The broad aim is to replicate the results from the influential paper, "A simple model of global cascades on random networks".

Watts, D. J. (2002). A simple model of global cascades on random networks. Proceedings of the National Academy of Sciences, 99(9), 5766-5771.

## Outline

Through this model, Watts demonstrated that a simple threshold rule could be used as a basis to explain global cascades, i.e., events where small initial random shocks spread to a significant proportion of the network. 

One rather surprising result from this study is that even when the average degree of the network is low, a global cascade can be triggered by randomly chosen seeds. This throws a spaner in the usual conception that large scale diffusions on a network are triggered only by "special people", i.e., influencers. 

## Model

A node in a graph is faced with a binary decision - to engage or to not engage (e.g., with new products or discussions with friends). They make this decision based on a simple rule - they compute the fraction of their neighbors that have engaged, compare it with their personal threshold, and engage if the fraction of engaged neighbors exceeds the threshold. In other words, the thought process is on the lines, "If at least 18% of my friends bought the new iPhone, I would want to buy it too".

The execution of this model is random and asynchronous, i.e., each node checks the status of its neighbors in a random order and decides whether to engage or not depending on the fraction of neighbors engaged at that point

In [4]:
for dep in ["LightGraphs", "Plots", "GR"]
    Pkg.add(dep)
end

INFO: Package LightGraphs is already installed
INFO: Package Plots is already installed
INFO: Package GR is already installed


In [12]:
using Distributions
using LightGraphs
using Plots
gr(); srand(20130810)

MersenneTwister(UInt32[0x01332bfa], Base.dSFMT.DSFMT_state(Int32[-1772545288, 1073534108, 1077066014, 1072915095, -2146195133, 1072843413, 301764553, 1073404181, 750472136, 1073628106  …  -1491411563, 1073194977, 716119449, 1072893711, 1632331784, 758890923, 1433693833, -13012230, 382, 0]), [1.45282, 1.84417, 1.69216, 1.64119, 1.06517, 1.11317, 1.83364, 1.8531, 1.51769, 1.38534  …  1.84606, 1.82905, 1.49949, 1.86043, 1.84268, 1.07144, 1.90694, 1.3571, 1.99488, 1.69479], 382)

In [7]:
function fraction_engaged(node::Int,
                          G::LightGraphs.SimpleGraphs.SimpleGraph,
                          node_status::BitVector)

    """
    Computes the fraction of neighbors engaged within the neighborhood
    of a given node. It uses the node status to check the engagement status of
    each of the nodes neighbors
    """
    num_engaged_neighbors = 0
    for nbr in neighbors(G, node)
        if node_status[nbr] == true
            num_engaged_neighbors += 1
        end
    end
    return num_engaged_neighbors/length(neighbors(G, node))
end

fraction_engaged (generic function with 1 method)

In [8]:
function update_node_status!(G::LightGraphs.SimpleGraphs.SimpleGraph,
                             node_status::BitVector,
                             threshold::Vector{Float64})
    """
    This function executes the random asynchronous updates of the entire network
    at each time step. In this conceptualization, each time step comprises mini
    time steps where a randomly shuffled node list updates at each iteration
    """
    
    for node in shuffle(vertices(G))
        if node_status[node] == false
            if fraction_engaged(node, G, node_status) > threshold[node]
                node_status[node] = true
            end
        end
    end

    return nothing
end

update_node_status! (generic function with 1 method)

In [9]:
function diffusion_simulation(n::Int,
                              z::Int,
                              threshold::Vector{Float64},
                              T::Int,
                              n_realizations::Int)
    """
    Executes the diffusion simulation.
    It creates a new Watts-Strogatz graph at each realization and seeds a single node
    (initialization). It then updates the network for the defined tnumber of time
    steps.

    The idea is to run the diffusion simulation a very large number
    of times and count the instances where we observe a
    global cascade, i.e., number of nodes engaged after the simulation
    process is a sizeable proportion of the network

    Hyper Parameters of the model
    ----------
    1. Number of nodes in the Watts-Strogatz graph (n)
    2. Average degree (z)
    3. Threshold (distribution or a specific value)
    4. Time steps for simulation to be run
    5. Number of realizations

    Output
    -----------
    A vector of number of engaged nodes at the end of each realization
    of the simulation

    Intended usage of results
    -----------
    Plot the cascades on the z-ϕ phase space; replicate results from
    "A simple model of global cascades on random networks", Watts (2002)
    """

    output = Vector{Int}(n_realizations)
    beta = z/n

    for r in 1:n_realizations
        G = watts_strogatz(n, z, beta)
        # Select a single random node from the network and seed it
        node_status = falses(nv(G))
        node_status[sample(vertices(G))] = true

        # Update the network for predefined number of time steps
        for _ in 1:T
            update_node_status!(G, node_status, threshold)
        end
        output[r] = sum(node_status)
    end

    return output

end

diffusion_simulation (generic function with 1 method)

## Results

We have two parameters that we wish to explore - the average degree of the network and the threshold for engagement. While we leave the full exploration of the parameter space to the reader, we demonstrate how this exploration could be done below. We take the example of two particular values of (z, $\phi$), i.e., (5, 0.18) and (6, 0.18) at which a phase transition occurs, i.e., at average network degree of 5 global cascades occur while at degree 6 no cascades occur irrespective of threshold!

In [14]:
const N = 10^4
z, threshold = 5, 0.18

@time data = diffusion_simulation(N, z, fill(threshold, N), 50, 100);

  4.194248 seconds (13.60 M allocations: 932.839 MiB, 4.40% gc time)


In [15]:
histogram(data); xlabel!("Number of engaged nodes"); ylabel!("Frequency"); plot!(legend = :none)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 Number of engaged nodes 
 
 
 Frequency

In [17]:
z, threshold = 6, 0.18

@time data = diffusion_simulation(N, z, fill(threshold, N), 50, 100);

  6.780746 seconds (20.39 M allocations: 1.191 GiB, 3.92% gc time)


In [18]:
histogram(data); xlabel!("Number of engaged nodes"); ylabel!("Frequency"); plot!(legend = :none)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.00 
 
 
 1.25 
 
 
 1.50 
 
 
 1.75 
 
 
 2.00 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 Number of engaged nodes 
 
 
 Frequency

As can be seen from these two graphs, no global cascade occurs in the second case, while there are a few in the first! It is remarkable that just increasing the average degree of the network by 1 changes the entire outcome of the diffusion process. The code presented in this notebook can be used to reproduce all the results discussed in Watts (2002) by executing the simulations at $z \in (1, 16)$ and $\phi \in (0.1, 0.26)$ 